# String Method Analysis Markov-State-Models
## Imports

In [ ]:
import os
import pickle
import sys
import logging
import numpy as np
import matplotlib.pyplot as plt
import glob
from math import ceil
from tqdm import tqdm


logging.getLogger("stringmethod").setLevel(logging.ERROR)
sys.path.append("../string-method-gmxapi/")
import src.analysis as spc

In [ ]:
%load_ext lab_black
%load_ext autoreload
%autoreload 2

In [ ]:
spc.jupyter_lab_notification()

In [ ]:
spc.jupyter_lab_error()

## Control Variables

In [ ]:
extract_data = False
if os.getenv("CALC_FES") == "True":
    calculate_s_path = True
    calculate_z_path = True
    calculate_path_errors = True
    calculate_converges = True
else:
    calculate_s_path = False
    calculate_z_path = False
    calculate_path_errors = False
    calculate_converges = False

In [ ]:
beads_per_iter = 18 - 2
swarms_per_bead = 32
steps_per_swarm = 2

## Load data

This notebook needs to run in the string simulation folder, this cell will get you there. You also set up a path for writing the figures.

In [ ]:
if os.getenv("NAME_SIM") == None:
    name_sim = "C2I_v1_amber/"  # Done
    name_sim = "C2I_lb_v1_amber/"  # Done
    name_sim = "C2I_lb_v1/"  # Done
    name_sim = "C2I_v1/"  # Done
else:
    name_sim = os.getenv("NAME_SIM")
path_raw = os.path.expanduser(f"~/Projects/string_sims/data/raw/{name_sim}")
path_interim = os.path.expanduser(f"~/Projects/string_sims/data/interim/{name_sim}")
path_processed = os.path.expanduser(f"~/Projects/string_sims/data/processed/{name_sim}")
path_XRD = os.path.expanduser(f"~/Projects/string_sims/models/raw_pdb/")
path_report = os.path.expanduser(f"~/Projects/string_sims/reports/figures/{name_sim}")
os.chdir(path_raw)
os.getcwd()

In [ ]:
with open("cv.pkl", "rb") as file:
    cvs, ndx_groups = pickle.load(file)

The `load_swarm_data` function will load the swarm data in the `cv_coordinates`. If you set `extract=True` it will read the data from the swarm files. If you have done this previously you can set `extract=False` so the function just reads `postprocessing/cv_coordinates.npy`. `first_iteration` can be used to exclude initial swarms as equilibration and `last_iteration` can be done to exclude some iterations for example if you want to estimate the FES convergence by comparing blocks of data.

In [ ]:
first_iteration = {
    "C2I_lb_v1/": 100,
    "C2I_v1/": 400,
    "C2I_v1_amber/": 100,
    "C2I_lb_v1_amber/": 100,
}
first_iteration = first_iteration[name_sim]

In [ ]:
if extract_data:
    cv_coordinates = spc.load_swarm_data(
        extract=True, first_iteration=first_iteration, last_iteration=None
    )
    np.save(f"{path_interim}cv_coordinates.npy", cv_coordinates)
else:
    cv_coordinates = np.load(f"{path_interim}cv_coordinates.npy")

In [ ]:
files = spc.natural_sort(glob.glob("./strings/string[0-9]*txt"))
strings = np.array([np.loadtxt(file).T for file in files])

Load pickle with data from XRD

In [ ]:
os.chdir(path_XRD)
with open(
    os.path.expanduser("~/Projects/string_sims/data/processed/XRD/XRD.pickle"), "rb"
) as handle:
    XRD_dictionary = pickle.load(handle)
os.chdir(path_raw)

Load MSM weights

In [ ]:
weights = np.load(f"{path_processed}weights.npy")
clusters = np.load(f"{path_processed}clusters.npy")

In [ ]:
F = np.load(f"{path_processed}FES_SF_IG.npy")
extent = np.load(f"{path_processed}extent.npy")

# Path CVs

Define path as the average of the last av_last_n_it of the strings. Obtain a reasonable guess of lambda.

In [ ]:
n_strings = strings.shape[0]
av_last_n_it = 25
path = np.mean(strings[n_strings - av_last_n_it :, :, :], axis=0)
lam = spc.get_path_lambda(path)
print(f"Default Lambda value for path {lam:.2f}")
lam = {
    "C2I_v1_amber/": 500,
    "C2I_v1/": spc.get_path_lambda(path),
    "C2I_lb_v1/": spc.get_path_lambda(path),
    "C2I_lb_v1_amber/": 100,
}
lam = lam[name_sim]
print(f"Lambda value for path {lam:.2f}")

Let's see if the lambda gives a well behaved path cv. The progress variable (s) should be increasing in the path itself and the distance to path variable (z) be low and constant.

In [ ]:
np.save(f"{path_processed}/path_lam{lam:.2f}.npy", path)

## Representation of Path CV

### Load data and calculate path cvs

In [ ]:
cv_of_path = []
for p in path.T:
    cv_of_path.append(spc.cvs_to_path(p, path=path, lam=lam))
cv_of_path = np.array(cv_of_path)

In [ ]:
cvs_path = []
for i in range(cv_coordinates.shape[0]):
    cvs_path.append([])
    for j in range(cv_coordinates.shape[1]):
        cvs_path[i].append(spc.cvs_to_path(cv_coordinates[i, j, :], path=path, lam=lam))
cvs_path = np.array(cvs_path)

In [ ]:
cv_proj = spc.cvs_to_SF_IG(cv_coordinates, [0, 1], [10, 11])
np.save(f"{path_interim}cv_proj.npy", cv_proj)

### Path CV on final string

Check how the path cv is represented in the average final string

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10 * 2, 7))
ax[0].plot(cv_of_path[:, 0], marker="o")
ax[1].plot(cv_of_path[:, 1], marker="o")
ax[0].set_xlabel("bead number")
ax[0].set_ylabel("S")
ax[1].set_xlabel("bead number")
ax[1].set_ylabel("Z")
ax[j].set_title(f"Path-variable {lam = :.2f}")
ax[1].set_title(f"Path-variable {lam = :.2f}")

### Path CV projected on IG vs SF

Check how the path variables project onto the canonical inactivation 2CV FES

In [ ]:
bandwidth = 0.05

In [ ]:
if calculate_s_path:
    s_of_cv, extent = spc.project_property_on_cv_kde(
        cv_proj, weights=weights, proper=cvs_path[:, :, 0:1], bandwidth=bandwidth
    )
    np.save(f"{path_processed}s_path_of_SF_IG.npy", s_of_cv)
else:
    s_of_cv = np.load(f"{path_processed}s_path_of_SF_IG.npy")

In [ ]:
if calculate_z_path:
    z_of_cv, extent = spc.project_property_on_cv_kde(
        cv_proj, weights=weights, proper=cvs_path[:, :, 1:2], bandwidth=bandwidth
    )
    np.save(f"{path_processed}z_path_of_SF_IG.npy", z_of_cv)
else:
    z_of_cv = np.load(f"{path_processed}z_path_of_SF_IG.npy")

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10 * 2, 7), sharex=True, sharey=True)
_ = spc.plot_2D_heatmap(
    s_of_cv,
    extent,
    cbar_label="s[path]",
    xlabel="SF (nm)",
    ylabel="IG (nm)",
    f_min=0,
    f_max=1,
    fig=fig,
    cmap=plt.cm.Spectral,
    ax=ax[0],
    n_colors=200,
    c_density=F,
    c_min=0,
    c_max=20,
    c_color="k",
)
ax[0].grid(None)
_ = spc.plot_2D_heatmap(
    z_of_cv,
    extent,
    cbar_label="z[path]",
    xlabel="SF (nm)",
    cmap=plt.cm.magma,
    f_min=0,
    f_max=0.15,
    fig=fig,
    ax=ax[1],
    n_colors=200,
    c_density=F,
    c_min=0,
    c_max=20,
    c_color="w",
)
ax[1].grid(None)
spc.add_XRD_values(XRD_dictionary, "SF", "IG", size=15, ax=ax[1], color="g")
spc.add_XRD_values(XRD_dictionary, "SF", "IG", size=15, ax=ax[0], color="g")
fig.tight_layout()
fig.savefig(path_report + "projection_path_cv.png")

## Calculate FES projected on path CV

Calculate FES preliminarily, for example to optimize `nbins` and `bandwidth`.

In [ ]:
s_path = cvs_path[:, :, 0:1]

In [ ]:
s_path_bandwidth = {
    "C2I_lb_v1/": 0.125,
    "C2I_v1/": 0.175,
    "C2I_v1_amber/": 0.2,
    "C2I_lb_v1_amber/": 0.3,
}
bandwidth = s_path_bandwidth[name_sim]

In [ ]:
%%time
nbins = 200
p_of_cv, extent = spc.get_kde(s_path, weights, bandwidth, nbins=nbins)
F0 = -np.log(p_of_cv)
F = F0 - F0.min()
F[F > 40] = np.nan
s = np.linspace(extent[0], extent[1], nbins)
np.save(f"{path_processed}FES_path.npy",np.vstack([s, F]))

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 7))
ax.plot(s, F, marker=".")
ax.legend()
ax.set_xlabel("s[path]", size=15)
ax.set_ylabel("F (kT)", size=15)
ax.set_ylim([0, 25])
ax.set_xlim([0, 1])

In [ ]:
e_val_s_path = np.sum(s_path.flatten() * weights)
print(f"The expected value of s_path is {e_val_s_path:.2f}")

## Calculate errors FES s-path

In [ ]:
%%time
import src.analysis as spc

blocks = [2, 4, 8, 16, 32]
n_blocks = len(blocks)
n_boot = 100
if calculate_path_errors:
    errors = spc.get_error(
        s_path,
        clusters,
        extent,
        n_boot=n_boot,
        bandwidth=bandwidth,
        nbin=nbins,
        blocks=blocks,
        seed=28101990,
        n_jobs=4,
    )
    np.save(f"{path_processed}path_errors_{n_boot}_{n_blocks}.npy", errors)
else:
    errors = np.load(f"{path_processed}path_errors_{n_boot}_{n_blocks}.npy")

Choose the number of blocks that gives you a high error.

Note,`e_min` and `e_max` are choosen to remove the extremely high or low values of error that are generated due to poor sampling or high free energy. These regions of the "free error surface" are not what we care about and thus we remove it from the statistic and the visualization.

In [ ]:
fig, ax = plt.subplots(1, 1)
error_block = 16
n_boot = 100
n_blocks = len(blocks)
s = np.linspace(extent[0], extent[1], nbins)
error = np.load(f"{path_processed}path_errors_{n_boot}_{n_blocks}.npy")
error = error[blocks.index(error_block), :]
ax.fill_between(s, F + error, F - error, alpha=0.3)
ax.plot(s, F, label=f"{n_boot=}", marker=".")
ax.legend()
ax.set_xlabel("s[path]", size=15)
ax.set_ylabel("F (kT)", size=15)
fig.savefig(path_report + "FES_path_cv.png")

### Study convergence of the FES as function of path

In [ ]:
k = {
    "C2I_lb_v1_amber/": 75,
    "C2I_lb_v1/": 30,
    "C2I_v1/": 50,
    "C2I_v1_amber/": 75,
}
k = k[name_sim]

In [ ]:
%%time
n_swarms = 36
n_beads = 18
step = 50
step = step * n_swarms * n_beads
bandwidth = s_path_bandwidth[name_sim]
final_factor = np.cov(s_path.flatten(), aweights=weights) * bandwidth

if calculate_converges:
    FES_vs_t = []
    FES_vs_t.append(np.linspace(extent[0], extent[1], nbins))
    for i in tqdm(range(step, s_path.shape[0]+step,step)):
        print(i)
        s = s_path[: i , :, :]
        c = cv_coordinates[: i, :, :]
        t = spc.cvs_to_tica(c, reversible=True) 
        cl = spc.k_means_cluster(t, k, stride=1, max_iter=500, n_jobs=4, seed=28101990)
        try:
            _, w= spc.get_msm(cl, n_jobs=4, reversible=True)
        except:
            print(f"didn't get msm for {i}")
            continue
        b = final_factor / np.cov(s_path.flatten(), aweights=weights)
        print(f"final factor {b}")
        p_of_cv, extent = spc.get_kde(
            s,
            w,
            b,
            nbins=nbins,
        )
        print(bandwidth*np.cov(s.flatten(), aweights=w))
        f0 = -np.log(p_of_cv)
        f = f0 - f0.min()
        f[f > 40] = np.nan
        FES_vs_t.append(f)
    FES_vs_t.append(F)
    FES_vs_t = np.array(FES_vs_t)
    np.save(f"{path_processed}FES_path_vs_t.npy", FES_vs_t)
else:
    FES_vs_t = np.load(f"{path_processed}FES_path_vs_t.npy")

In [ ]:
fig, ax = spc.plot_FES_1d_vs_t(FES_vs_t[:, :], xlabel="s[path] (nm)", error=error)
ax.set_ylim([0, 25])
ax.set_xlim([0, 1])
fig.savefig(path_report + "FES_path_convergence.png")

## FES s-path vs property

In [ ]:
s_path = cvs_path[:, :, 0:1]

## Path vs IG and SF (checks)

### SF

In [ ]:
other_cv_id = [0, 1]
other_cv = spc.average_strings_to_cv(cv_coordinates, other_cv_id)
cv_name = "SF"
cv_fig_label = "SF (nm)"
f_max = 25
bandwidth = 0.05
save_npy = f"{path_processed}FES_{cv_name}_path.npy"
save_fig = path_report + f"FES_{cv_name}.png"
save_extent = f"{path_processed}/extent_{cv_name}_path.npy"
spc.plot_path_vs_cv_FES(
    s_path,
    other_cv,
    weights,
    cv_name,
    cv_fig_label,
    bandwidth,
    f_max,
    save_npy,
    save_fig,
    save_extent,
)

Do the projection and take log. You have to choose a bandwidth for the [KDE](https://en.wikipedia.org/wiki/Kernel_density_estimation) of the histogram. It should be big enough to reduce noise but not so big to remove features. If you give `None`

### IG

In [ ]:
other_cv_id = [10, 11]
cv_name = "IG"
cv_fig_label = "IG (nm)"
other_cv = spc.average_strings_to_cv(cv_coordinates, other_cv_id)
f_max = 25
bandwidth = 0.05
save_npy = f"{path_processed}FES_{cv_name}_path.npy"
save_fig = path_report + f"FES_{cv_name}.png"
save_extent = f"{path_processed}/extent_{cv_name}.npy"
save_extent = f"{path_processed}/extent_{cv_name}_path.npy"
spc.plot_path_vs_cv_FES(
    s_path,
    other_cv,
    weights,
    cv_name,
    cv_fig_label,
    bandwidth,
    f_max,
    save_npy,
    save_fig,
    save_extent,
)

## Path vs L81-W67

In [ ]:
other_cv_id = [32, 33, 34, 35]
cv_name = "L81-W67"
cv_fig_label = "L81-W67 (nm)"
other_cv = spc.average_strings_to_cv(cv_coordinates, other_cv_id)
f_max = 25
bandwidth = 0.05
save_npy = f"{path_processed}FES_{cv_name}_path.npy"
save_fig = path_report + f"FES_{cv_name}.png"
save_extent = f"{path_processed}/extent_{cv_name}_path.npy"
spc.plot_path_vs_cv_FES(
    s_path,
    other_cv,
    weights,
    cv_name,
    cv_fig_label,
    bandwidth,
    f_max,
    save_npy,
    save_fig,
    save_extent,
)

## Path vs L81-W67 0

In [ ]:
other_cv_id = 32
cv_name = "L81-W67_0"
cv_fig_label = "L81-W67 0th (nm)"
other_cv = cv_coordinates[:, :, other_cv_id : other_cv_id + 1]
f_max = 25
bandwidth = 0.05
save_npy = f"{path_processed}FES_{cv_name}_path.npy"
save_fig = path_report + f"FES_{cv_name}.png"
save_extent = f"{path_processed}/extent_{cv_name}_path.npy"
spc.plot_path_vs_cv_FES(
    s_path,
    other_cv,
    weights,
    cv_name,
    cv_fig_label,
    bandwidth,
    f_max,
    save_npy,
    save_fig,
    save_extent,
)

## Path vs L81-W67 1

In [ ]:
other_cv_id = 33
cv_name = "L81-W67_1"
cv_fig_label = "L81-W67 1th (nm)"
other_cv = cv_coordinates[:, :, other_cv_id : other_cv_id + 1]
f_max = 25
bandwidth = 0.05
save_npy = f"{path_processed}FES_{cv_name}_path.npy"
save_fig = path_report + f"FES_{cv_name}.png"
save_extent = f"{path_processed}/extent_{cv_name}_path.npy"
spc.plot_path_vs_cv_FES(
    s_path,
    other_cv,
    weights,
    cv_name,
    cv_fig_label,
    bandwidth,
    f_max,
    save_npy,
    save_fig,
    save_extent,
)

## Path vs L81-W67 2

In [ ]:
other_cv_id = 34
cv_name = "L81-W67_2"
cv_fig_label = "L81-W67 2th (nm)"
other_cv = cv_coordinates[:, :, other_cv_id : other_cv_id + 1]
f_max = 25
bandwidth = 0.05
save_npy = f"{path_processed}FES_{cv_name}_path.npy"
save_fig = path_report + f"FES_{cv_name}.png"
save_extent = f"{path_processed}/extent_{cv_name}_path.npy"
spc.plot_path_vs_cv_FES(
    s_path,
    other_cv,
    weights,
    cv_name,
    cv_fig_label,
    bandwidth,
    f_max,
    save_npy,
    save_fig,
    save_extent,
)

## Path vs L81-W67 3

In [ ]:
other_cv_id = 35
cv_name = "L81-W67_3"
cv_fig_label = "L81-W67 3th (nm)"
other_cv = cv_coordinates[:, :, other_cv_id : other_cv_id + 1]
f_max = 25
bandwidth = 0.05
save_npy = f"{path_processed}FES_{cv_name}_path.npy"
save_fig = path_report + f"FES_{cv_name}.png"
save_extent = f"{path_processed}/extent_{cv_name}_path.npy"
spc.plot_path_vs_cv_FES(
    s_path,
    other_cv,
    weights,
    cv_name,
    cv_fig_label,
    bandwidth,
    f_max,
    save_npy,
    save_fig,
    save_extent,
)

## Path vs I100-F103 (checks)

In [ ]:
other_cv_id = [12, 13, 14, 15]
cv_name = "I100-F103"
cv_fig_label = "I100-F103 (nm)"
other_cv = spc.average_strings_to_cv(cv_coordinates, other_cv_id)
f_max = 25
bandwidth = 0.05
save_npy = f"{path_processed}FES_{cv_name}_path.npy"
save_fig = path_report + f"FES_{cv_name}.png"
save_extent = f"{path_processed}/extent_{cv_name}_path.npy"
spc.plot_path_vs_cv_FES(
    s_path,
    other_cv,
    weights,
    cv_name,
    cv_fig_label,
    bandwidth,
    f_max,
    save_npy,
    save_fig,
    save_extent,
)

## Path vs F114-A32 TWIST

In [ ]:
other_cv_id = [16, 17, 18, 19]
cv_name = "F114-A32"
cv_fig_label = "F114-A32 (nm)"
other_cv = spc.average_strings_to_cv(cv_coordinates, other_cv_id)
f_max = 25
bandwidth = 0.05
save_npy = f"{path_processed}FES_{cv_name}_path.npy"
save_fig = path_report + f"FES_{cv_name}.png"
save_extent = f"{path_processed}/extent_{cv_name}_path.npy"
spc.plot_path_vs_cv_FES(
    s_path,
    other_cv,
    weights,
    cv_name,
    cv_fig_label,
    bandwidth,
    f_max,
    save_npy,
    save_fig,
    save_extent,
)

## Path vs E118-A28 TWIST

In [ ]:
other_cv_id = [24, 25, 26, 27]
cv_name = "E118-A28"
cv_fig_label = "E118-A28 (nm)"
other_cv = spc.average_strings_to_cv(cv_coordinates, other_cv_id)
f_max = 25
bandwidth = 0.05
save_npy = f"{path_processed}FES_{cv_name}_path.npy"
save_fig = path_report + f"FES_{cv_name}.png"
save_extent = f"{path_processed}/extent_{cv_name}_path.npy"
spc.plot_path_vs_cv_FES(
    s_path,
    other_cv,
    weights,
    cv_name,
    cv_fig_label,
    bandwidth,
    f_max,
    save_npy,
    save_fig,
    save_extent,
)

## Path vs I100 $\chi_1$

In [ ]:
cv_name = "chi1_I100"
cv_fig_label = "$\chi_1$ I100 ($\deg$)"
other_cv = np.load(f"{path_interim}/{cv_name}.npy")[:, :]
other_cv = other_cv.reshape([other_cv.shape[0] // 2, 2, 1])
f_max = 25
bandwidth = 0.05
save_npy = f"{path_processed}FES_{cv_name}_path.npy"
save_fig = path_report + f"FES_{cv_name}.png"
save_extent = f"{path_processed}/extent_{cv_name}_path.npy"
spc.plot_path_vs_cv_FES(
    s_path,
    other_cv,
    weights,
    cv_name,
    cv_fig_label,
    bandwidth,
    f_max,
    save_npy,
    save_fig,
    save_extent,
)

## Path vs F103 $\chi_1$

In [ ]:
cv_name = "chi1_F103"
cv_fig_label = "$\chi_1$ F103 ($\deg$)"
other_cv = np.load(f"{path_interim}/{cv_name}.npy")[:, :]
other_cv = other_cv.reshape([other_cv.shape[0] // 2, 2, 1])
f_max = 25
bandwidth = 0.05
save_npy = f"{path_processed}FES_{cv_name}_path.npy"
save_fig = path_report + f"FES_{cv_name}.png"
save_extent = f"{path_processed}/extent_{cv_name}_path.npy"
spc.plot_path_vs_cv_FES(
    s_path,
    other_cv,
    weights,
    cv_name,
    cv_fig_label,
    bandwidth,
    f_max,
    save_npy,
    save_fig,
    save_extent,
)

## Path vs E71 $\chi_1$

In [ ]:
cv_name = "chi1_E71"
cv_fig_label = "$\chi_1$ E71 ($\deg$)"
other_cv = np.load(f"{path_interim}/{cv_name}.npy")[:, :]
other_cv = other_cv.reshape([other_cv.shape[0] // 2, 2, 1])
f_max = 25
bandwidth = 0.05
save_npy = f"{path_processed}FES_{cv_name}_path.npy"
save_fig = path_report + f"FES_{cv_name}.png"
save_extent = f"{path_processed}/extent_{cv_name}_path.npy"
spc.plot_path_vs_cv_FES(
    s_path,
    other_cv,
    weights,
    cv_name,
    cv_fig_label,
    bandwidth,
    f_max,
    save_npy,
    save_fig,
    save_extent,
)

## Path vs E72 $\chi_2$

In [ ]:
cv_name = "chi2_E71"
cv_fig_label = "$\chi_2$ E71 ($\deg$)"
other_cv = np.load(f"{path_interim}/{cv_name}.npy")[:, :]
other_cv = other_cv.reshape([other_cv.shape[0] // 2, 2, 1])
f_max = 25
bandwidth = 0.05
save_npy = f"{path_processed}FES_{cv_name}_path.npy"
save_fig = path_report + f"FES_{cv_name}.png"
save_extent = f"{path_processed}/extent_{cv_name}_path.npy"
spc.plot_path_vs_cv_FES(
    s_path,
    other_cv,
    weights,
    cv_name,
    cv_fig_label,
    bandwidth,
    f_max,
    save_npy,
    save_fig,
    save_extent,
)

## Path vs Y82 $\chi_1$

In [ ]:
cv_name = "chi1_Y82"
cv_fig_label = "$\chi_1$ Y82 ($\deg$)"
other_cv = np.load(f"{path_interim}/{cv_name}.npy")[:, :]
other_cv = np.mean(other_cv, axis=1, keepdims=True)
other_cv = other_cv.reshape([other_cv.shape[0] // 2, 2, 1])
f_max = 25
bandwidth = 0.05
save_npy = f"{path_processed}FES_{cv_name}_path.npy"
save_fig = path_report + f"FES_{cv_name}.png"
save_extent = f"{path_processed}/extent_{cv_name}_path.npy"
spc.plot_path_vs_cv_FES(
    s_path,
    other_cv,
    weights,
    cv_name,
    cv_fig_label,
    bandwidth,
    f_max,
    save_npy,
    save_fig,
    save_extent,
)

### SU 0

In [ ]:
cv_name = "chi1_Y82"
cv_fig_label = "$\chi_1$ Y82 SU0 ($\deg$)"
other_cv = np.load(f"{path_interim}/{cv_name}.npy")[:, 0]
other_cv = other_cv.reshape([other_cv.shape[0] // 2, 2, 1])
f_max = 25
bandwidth = 0.05
save_npy = f"{path_processed}FES_{cv_name}_path.npy"
save_fig = path_report + f"FES_{cv_name}.png"
save_extent = f"{path_processed}/extent_{cv_name}_path.npy"
spc.plot_path_vs_cv_FES(
    s_path,
    other_cv,
    weights,
    cv_name,
    cv_fig_label,
    bandwidth,
    f_max,
    save_npy,
    save_fig,
    save_extent,
)

### SU 1

In [ ]:
cv_name = "chi1_Y82"
cv_fig_label = "$\chi_1$ Y82 SU1 ($\deg$)"
other_cv = np.load(f"{path_interim}/{cv_name}.npy")[:, 1]
other_cv = other_cv.reshape([other_cv.shape[0] // 2, 2, 1])
f_max = 25
bandwidth = 0.05
save_npy = f"{path_processed}FES_{cv_name}_path.npy"
save_fig = path_report + f"FES_{cv_name}.png"
save_extent = f"{path_processed}/extent_{cv_name}_path.npy"
spc.plot_path_vs_cv_FES(
    s_path,
    other_cv,
    weights,
    cv_name,
    cv_fig_label,
    bandwidth,
    f_max,
    save_npy,
    save_fig,
    save_extent,
)

### SU 2

In [ ]:
cv_name = "chi1_Y82"
cv_fig_label = "$\chi_1$ Y82 SU2 ($\deg$)"
other_cv = np.load(f"{path_interim}/{cv_name}.npy")[:, 2]
other_cv = other_cv.reshape([other_cv.shape[0] // 2, 2, 1])
f_max = 25
bandwidth = 0.05
save_npy = f"{path_processed}FES_{cv_name}_path.npy"
save_fig = path_report + f"FES_{cv_name}.png"
save_extent = f"{path_processed}/extent_{cv_name}_path.npy"
spc.plot_path_vs_cv_FES(
    s_path,
    other_cv,
    weights,
    cv_name,
    cv_fig_label,
    bandwidth,
    f_max,
    save_npy,
    save_fig,
    save_extent,
)

### SU 3

In [ ]:
cv_name = "chi1_Y82"
cv_fig_label = "$\chi_1$ Y82 SU3 ($\deg$)"
other_cv = np.load(f"{path_interim}/{cv_name}.npy")[:, 3]
other_cv = other_cv.reshape([other_cv.shape[0] // 2, 2, 1])
f_max = 25
bandwidth = 0.05
save_npy = f"{path_processed}FES_{cv_name}_path.npy"
save_fig = path_report + f"FES_{cv_name}.png"
save_extent = f"{path_processed}/extent_{cv_name}_path.npy"
spc.plot_path_vs_cv_FES(
    s_path,
    other_cv,
    weights,
    cv_name,
    cv_fig_label,
    bandwidth,
    f_max,
    save_npy,
    save_fig,
    save_extent,
)

## Path vs Y82 $\chi_2$

In [ ]:
cv_name = "chi2_Y82"
cv_fig_label = "$\chi_2$ Y82 ($\deg$)"
other_cv = np.load(f"{path_interim}/{cv_name}.npy")[:, :]
other_cv = np.mean(other_cv, axis=1, keepdims=True)
other_cv = other_cv.reshape([other_cv.shape[0] // 2, 2, 1])
f_max = 25
bandwidth = 0.05
save_npy = f"{path_processed}FES_{cv_name}_path.npy"
save_fig = path_report + f"FES_{cv_name}.png"
save_extent = f"{path_processed}/extent_{cv_name}_path.npy"
spc.plot_path_vs_cv_FES(
    s_path,
    other_cv,
    weights,
    cv_name,
    cv_fig_label,
    bandwidth,
    f_max,
    save_npy,
    save_fig,
    save_extent,
)

### SU 0

In [ ]:
cv_name = "chi2_Y82"
cv_fig_label = "$\chi_2$ Y82 SU0 ($\deg$)"
other_cv = np.load(f"{path_interim}/{cv_name}.npy")[:, 0]
other_cv = other_cv.reshape([other_cv.shape[0] // 2, 2, 1])
f_max = 25
bandwidth = 0.05
save_npy = f"{path_processed}FES_{cv_name}_path.npy"
save_fig = path_report + f"FES_{cv_name}.png"
save_extent = f"{path_processed}/extent_{cv_name}_path.npy"
spc.plot_path_vs_cv_FES(
    s_path,
    other_cv,
    weights,
    cv_name,
    cv_fig_label,
    bandwidth,
    f_max,
    save_npy,
    save_fig,
    save_extent,
)

### SU 1

In [ ]:
cv_name = "chi2_Y82"
cv_fig_label = "$\chi_2$ Y82 SU1 ($\deg$)"
other_cv = np.load(f"{path_interim}/{cv_name}.npy")[:, 1]
other_cv = other_cv.reshape([other_cv.shape[0] // 2, 2, 1])
f_max = 25
bandwidth = 0.05
save_npy = f"{path_processed}FES_{cv_name}_path.npy"
save_fig = path_report + f"FES_{cv_name}.png"
save_extent = f"{path_processed}/extent_{cv_name}_path.npy"
spc.plot_path_vs_cv_FES(
    s_path,
    other_cv,
    weights,
    cv_name,
    cv_fig_label,
    bandwidth,
    f_max,
    save_npy,
    save_fig,
    save_extent,
)

### SU 2

In [ ]:
cv_name = "chi2_Y82"
cv_fig_label = "$\chi_2$ Y82 SU2 ($\deg$)"
other_cv = np.load(f"{path_interim}/{cv_name}.npy")[:, 2]
other_cv = other_cv.reshape([other_cv.shape[0] // 2, 2, 1])
f_max = 25
bandwidth = 0.05
save_npy = f"{path_processed}FES_{cv_name}_path.npy"
save_fig = path_report + f"FES_{cv_name}.png"
save_extent = f"{path_processed}/extent_{cv_name}_path.npy"
spc.plot_path_vs_cv_FES(
    s_path,
    other_cv,
    weights,
    cv_name,
    cv_fig_label,
    bandwidth,
    f_max,
    save_npy,
    save_fig,
    save_extent,
)

### SU 3

In [ ]:
cv_name = "chi2_Y82"
cv_fig_label = "$\chi_2$ Y82 SU3 ($\deg$)"
other_cv = np.load(f"{path_interim}/{cv_name}.npy")[:, 3]
other_cv = other_cv.reshape([other_cv.shape[0] // 2, 2, 1])
f_max = 25
bandwidth = 0.05
save_npy = f"{path_processed}FES_{cv_name}_path.npy"
save_fig = path_report + f"FES_{cv_name}.png"
save_extent = f"{path_processed}/extent_{cv_name}_path.npy"
spc.plot_path_vs_cv_FES(
    s_path,
    other_cv,
    weights,
    cv_name,
    cv_fig_label,
    bandwidth,
    f_max,
    save_npy,
    save_fig,
    save_extent,
)

## Path vs T74-F103

In [ ]:
cv_name = "dis_T74-F103"
cv_fig_label = "T74-F103 (nm)"
other_cv = np.load(f"{path_interim}/{cv_name}.npy")[:, :]
other_cv = other_cv.reshape([other_cv.shape[0] // 2, 2, 1])
f_max = 25
bandwidth = 0.05
save_npy = f"{path_processed}FES_{cv_name}_path.npy"
save_fig = path_report + f"FES_{cv_name}.png"
save_extent = f"{path_processed}/extent_{cv_name}_path.npy"
spc.plot_path_vs_cv_FES(
    s_path,
    other_cv,
    weights,
    cv_name,
    cv_fig_label,
    bandwidth,
    f_max,
    save_npy,
    save_fig,
    save_extent,
)

## Path vs T74-I100

In [ ]:
cv_name = "dis_T74-I100"
cv_fig_label = "T74-I100 (nm)"
other_cv = np.load(f"{path_interim}/{cv_name}.npy")[:, :]
other_cv = other_cv.reshape([other_cv.shape[0] // 2, 2, 1])
f_max = 25
bandwidth = 0.05
save_npy = f"{path_processed}FES_{cv_name}_path.npy"
save_fig = path_report + f"FES_{cv_name}.png"
save_extent = f"{path_processed}/extent_{cv_name}_path.npy"
spc.plot_path_vs_cv_FES(
    s_path,
    other_cv,
    weights,
    cv_name,
    cv_fig_label,
    bandwidth,
    f_max,
    save_npy,
    save_fig,
    save_extent,
)

## Path vs P-R64

In [ ]:
cv_name = "dis_P-R64"
cv_fig_label = "P-R64 (nm)"
other_cv = np.load(f"{path_interim}/{cv_name}.npy")[:, :]
other_cv = other_cv.reshape([other_cv.shape[0] // 2, 2, 1])
f_max = 25
bandwidth = 0.05
save_npy = f"{path_processed}FES_{cv_name}_path.npy"
save_fig = path_report + f"FES_{cv_name}.png"
save_extent = f"{path_processed}/extent_{cv_name}_path.npy"
spc.plot_path_vs_cv_FES(
    s_path,
    other_cv,
    weights,
    cv_name,
    cv_fig_label,
    bandwidth,
    f_max,
    save_npy,
    save_fig,
    save_extent,
)

## Path vs P-R89

In [ ]:
cv_name = "dis_P-R89"
cv_fig_label = "P-R89 (nm)"
other_cv = np.load(f"{path_interim}/{cv_name}.npy")[:, :]
other_cv = other_cv.reshape([other_cv.shape[0] // 2, 2, 1])
f_max = 25
bandwidth = 0.05
save_npy = f"{path_processed}FES_{cv_name}_path.npy"
save_fig = path_report + f"FES_{cv_name}.png"
save_extent = f"{path_processed}/extent_{cv_name}_path.npy"
spc.plot_path_vs_cv_FES(
    s_path,
    other_cv,
    weights,
    cv_name,
    cv_fig_label,
    bandwidth,
    f_max,
    save_npy,
    save_fig,
    save_extent,
)

## Path vs E71-G79

In [ ]:
cv_name = "dis_E71-G79"
cv_fig_label = "E71-G79 (nm)"
other_cv = np.load(f"{path_interim}/{cv_name}.npy")[:, :]
other_cv = other_cv.reshape([other_cv.shape[0] // 2, 2, 1])
f_max = 25
bandwidth = 0.05
save_npy = f"{path_processed}FES_{cv_name}_path.npy"
save_fig = path_report + f"FES_{cv_name}.png"
save_extent = f"{path_processed}/extent_{cv_name}_path.npy"
spc.plot_path_vs_cv_FES(
    s_path,
    other_cv,
    weights,
    cv_name,
    cv_fig_label,
    bandwidth,
    f_max,
    save_npy,
    save_fig,
    save_extent,
)

## Path vs E71 angle

In [ ]:
cv_name = "is_E71_horizontal"
cv_fig_label = "E71 angle (deg)"
other_cv = np.load(f"{path_interim}/{cv_name}.npy")[:, :]
other_cv = other_cv.reshape([other_cv.shape[0] // 2, 2, 1])
f_max = 25
bandwidth = 0.05
save_npy = f"{path_processed}FES_{cv_name}_path.npy"
save_fig = path_report + f"FES_{cv_name}.png"
save_extent = f"{path_processed}/extent_{cv_name}_path.npy"
spc.plot_path_vs_cv_FES(
    s_path,
    other_cv,
    weights,
    cv_name,
    cv_fig_label,
    bandwidth,
    f_max,
    save_npy,
    save_fig,
    save_extent,
)

## Path vs E71 angle PROB

In [ ]:
cv_name = "is_E71_horizontal_B"
cv_fig_label = "E71 angle (deg)"
other_cv = np.load(f"{path_interim}/{cv_name}.npy")[:, :]
other_cv = other_cv.reshape([other_cv.shape[0] // 2, 2, 1])
f_max = 25
bandwidth = 0.05
save_npy = f"{path_processed}FES_{cv_name}_path.npy"
save_fig = path_report + f"FES_{cv_name}.png"
save_extent = f"{path_processed}/extent_{cv_name}_path.npy"
spc.plot_path_vs_cv_FES(
    s_path,
    other_cv,
    weights,
    cv_name,
    cv_fig_label,
    bandwidth,
    f_max,
    save_npy,
    save_fig,
    save_extent,
)

## Path vs E71 angle PROD

In [ ]:
cv_name = "is_E71_horizontal_D"
cv_fig_label = "E71 angle (deg)"
other_cv = np.load(f"{path_interim}/{cv_name}.npy")[:, :]
other_cv = other_cv.reshape([other_cv.shape[0] // 2, 2, 1])
f_max = 25
bandwidth = 0.05
save_npy = f"{path_processed}FES_{cv_name}_path.npy"
save_fig = path_report + f"FES_{cv_name}.png"
save_extent = f"{path_processed}/extent_{cv_name}_path.npy"
spc.plot_path_vs_cv_FES(
    s_path,
    other_cv,
    weights,
    cv_name,
    cv_fig_label,
    bandwidth,
    f_max,
    save_npy,
    save_fig,
    save_extent,
)

## Path vs W67-D80

### Mean

In [ ]:
cv_name = "dis_W67-D80"
cv_fig_label = "W67-D80 (nm)"
other_cv = np.load(f"{path_interim}/{cv_name}.npy")[:, :]
other_cv = np.mean(other_cv, axis=1, keepdims=True)
other_cv = other_cv.reshape([other_cv.shape[0] // 2, 2, 1])
f_max = 25
bandwidth = 0.05
save_npy = f"{path_processed}FES_{cv_name}_path.npy"
save_fig = path_report + f"FES_{cv_name}.png"
save_extent = f"{path_processed}/extent_{cv_name}_path.npy"
spc.plot_path_vs_cv_FES(
    s_path,
    other_cv,
    weights,
    cv_name,
    cv_fig_label,
    bandwidth,
    f_max,
    save_npy,
    save_fig,
    save_extent,
)

### 0th SU

In [ ]:
cv_name = "dis_W67-D80"
n_su = 0
cv_fig_label = f"W67-D80 {n_su}th SU (nm)"
other_cv = np.load(f"{path_interim}/{cv_name}.npy")[:, :]
other_cv = other_cv[:, n_su : n_su + 1]
other_cv = other_cv.reshape([other_cv.shape[0] // 2, 2, 1])
f_max = 25
bandwidth = 0.05
save_npy = f"{path_processed}FES_{cv_name}_path_{n_su}.npy"
save_fig = path_report + f"FES_{cv_name}_{n_su}.png"
save_extent = f"{path_processed}/extent_{cv_name}_path.npy"
spc.plot_path_vs_cv_FES(
    s_path,
    other_cv,
    weights,
    cv_name,
    cv_fig_label,
    bandwidth,
    f_max,
    save_npy,
    save_fig,
    save_extent,
)

### 1th SU

In [ ]:
cv_name = "dis_W67-D80"
n_su = 1
cv_fig_label = f"W67-D80 {n_su}th SU (nm)"
other_cv = np.load(f"{path_interim}/{cv_name}.npy")[:, :]
other_cv = other_cv[:, n_su : n_su + 1]
other_cv = other_cv.reshape([other_cv.shape[0] // 2, 2, 1])
f_max = 25
bandwidth = 0.05
save_npy = f"{path_processed}FES_{cv_name}_path_{n_su}.npy"
save_fig = path_report + f"FES_{cv_name}_{n_su}.png"
save_extent = f"{path_processed}/extent_{cv_name}_path.npy"
spc.plot_path_vs_cv_FES(
    s_path,
    other_cv,
    weights,
    cv_name,
    cv_fig_label,
    bandwidth,
    f_max,
    save_npy,
    save_fig,
    save_extent,
)

### 2th SU

In [ ]:
cv_name = "dis_W67-D80"
n_su = 2
cv_fig_label = f"W67-D80 {n_su}th SU (nm)"
other_cv = np.load(f"{path_interim}/{cv_name}.npy")[:, :]
other_cv = other_cv[:, n_su : n_su + 1]
other_cv = other_cv.reshape([other_cv.shape[0] // 2, 2, 1])
f_max = 25
bandwidth = 0.05
save_npy = f"{path_processed}FES_{cv_name}_path_{n_su}.npy"
save_fig = path_report + f"FES_{cv_name}_{n_su}.png"
save_extent = f"{path_processed}/extent_{cv_name}_path.npy"
spc.plot_path_vs_cv_FES(
    s_path,
    other_cv,
    weights,
    cv_name,
    cv_fig_label,
    bandwidth,
    f_max,
    save_npy,
    save_fig,
    save_extent,
)

### 3th SU

In [ ]:
cv_name = "dis_W67-D80"
n_su = 3
cv_fig_label = f"W67-D80 {n_su}th SU (nm)"
other_cv = np.load(f"{path_interim}/{cv_name}.npy")[:, :]
other_cv = other_cv[:, n_su : n_su + 1]
other_cv = other_cv.reshape([other_cv.shape[0] // 2, 2, 1])
f_max = 25
bandwidth = 0.05
save_npy = f"{path_processed}FES_{cv_name}_path_{n_su}.npy"
save_fig = path_report + f"FES_{cv_name}_{n_su}.png"
save_extent = f"{path_processed}/extent_{cv_name}_path.npy"
spc.plot_path_vs_cv_FES(
    s_path,
    other_cv,
    weights,
    cv_name,
    cv_fig_label,
    bandwidth,
    f_max,
    save_npy,
    save_fig,
    save_extent,
)

## Path vs E71-D80

### Mean

In [ ]:
cv_name = "dis_E71-D80"
cv_fig_label = "E71-D80 (nm)"
other_cv = np.load(f"{path_interim}/{cv_name}.npy")[:, :]
other_cv = np.mean(other_cv, axis=1, keepdims=True)
other_cv = other_cv.reshape([other_cv.shape[0] // 2, 2, 1])
f_max = 25
bandwidth = 0.05
save_npy = f"{path_processed}FES_{cv_name}_path.npy"
save_fig = path_report + f"FES_{cv_name}.png"
save_extent = f"{path_processed}/extent_{cv_name}_path.npy"
spc.plot_path_vs_cv_FES(
    s_path,
    other_cv,
    weights,
    cv_name,
    cv_fig_label,
    bandwidth,
    f_max,
    save_npy,
    save_fig,
    save_extent,
)

### 0th SU

In [ ]:
cv_name = "dis_E71-D80"
n_su = 0
cv_fig_label = f"E71-D80 {n_su}th SU (nm)"
other_cv = np.load(f"{path_interim}/{cv_name}.npy")[:, :]
other_cv = other_cv[:, n_su : n_su + 1]
other_cv = other_cv.reshape([other_cv.shape[0] // 2, 2, 1])
f_max = 35
bandwidth = 0.05
save_npy = f"{path_processed}FES_{cv_name}_path_{n_su}.npy"
save_fig = path_report + f"FES_{cv_name}_{n_su}.png"
save_extent = f"{path_processed}/extent_{cv_name}_path.npy"
spc.plot_path_vs_cv_FES(
    s_path,
    other_cv,
    weights,
    cv_name,
    cv_fig_label,
    bandwidth,
    f_max,
    save_npy,
    save_fig,
    save_extent,
)

### 1th SU

In [ ]:
cv_name = "dis_E71-D80"
n_su = 1
cv_fig_label = f"E71-D80 {n_su}th SU (nm)"
other_cv = np.load(f"{path_interim}/{cv_name}.npy")[:, :]
other_cv = other_cv[:, n_su : n_su + 1]
other_cv = other_cv.reshape([other_cv.shape[0] // 2, 2, 1])
f_max = 35
bandwidth = 0.05
save_npy = f"{path_processed}FES_{cv_name}_path_{n_su}.npy"
save_fig = path_report + f"FES_{cv_name}_{n_su}.png"
save_extent = f"{path_processed}/extent_{cv_name}_path.npy"
spc.plot_path_vs_cv_FES(
    s_path,
    other_cv,
    weights,
    cv_name,
    cv_fig_label,
    bandwidth,
    f_max,
    save_npy,
    save_fig,
    save_extent,
)

### 2th SU

In [ ]:
cv_name = "dis_E71-D80"
n_su = 2
cv_fig_label = f"E71-D80 {n_su}th SU (nm)"
other_cv = np.load(f"{path_interim}/{cv_name}.npy")[:, :]
other_cv = other_cv[:, n_su : n_su + 1]
other_cv = other_cv.reshape([other_cv.shape[0] // 2, 2, 1])
f_max = 35
bandwidth = 0.05
save_npy = f"{path_processed}FES_{cv_name}_path_{n_su}.npy"
save_fig = path_report + f"FES_{cv_name}_{n_su}.png"
save_extent = f"{path_processed}/extent_{cv_name}_path.npy"
spc.plot_path_vs_cv_FES(
    s_path,
    other_cv,
    weights,
    cv_name,
    cv_fig_label,
    bandwidth,
    f_max,
    save_npy,
    save_fig,
    save_extent,
)

### 3th SU

In [ ]:
cv_name = "dis_E71-D80"
n_su = 3
cv_fig_label = f"E71-D80 {n_su}th SU (nm)"
other_cv = np.load(f"{path_interim}/{cv_name}.npy")[:, :]
other_cv = other_cv[:, n_su : n_su + 1]
other_cv = other_cv.reshape([other_cv.shape[0] // 2, 2, 1])
f_max = 25
bandwidth = 0.05
save_npy = f"{path_processed}FES_{cv_name}_path_{n_su}.npy"
save_fig = path_report + f"FES_{cv_name}_{n_su}.png"
save_extent = f"{path_processed}/extent_{cv_name}_path.npy"
spc.plot_path_vs_cv_FES(
    s_path,
    other_cv,
    weights,
    cv_name,
    cv_fig_label,
    bandwidth,
    f_max,
    save_npy,
    save_fig,
    save_extent,
)

## Path vs R89-D80

### Mean

In [ ]:
cv_name = "dis_R89-D80"
cv_fig_label = "R89-D80 (nm)"
other_cv = np.load(f"{path_interim}/{cv_name}.npy")[:, :]
other_cv = np.mean(other_cv, axis=1, keepdims=True)
other_cv = other_cv.reshape([other_cv.shape[0] // 2, 2, 1])
f_max = 25
bandwidth = 0.05
save_npy = f"{path_processed}FES_{cv_name}_path.npy"
save_fig = path_report + f"FES_{cv_name}.png"
save_extent = f"{path_processed}/extent_{cv_name}_path.npy"
spc.plot_path_vs_cv_FES(
    s_path,
    other_cv,
    weights,
    cv_name,
    cv_fig_label,
    bandwidth,
    f_max,
    save_npy,
    save_fig,
    save_extent,
)

## Path vs G79-Y82

### Mean

In [ ]:
cv_name = "dis_G79-Y82"
cv_fig_label = "G79-Y82 (nm)"
other_cv = np.load(f"{path_interim}/{cv_name}.npy")[:, :]
other_cv = np.mean(other_cv, axis=1, keepdims=True)
other_cv = other_cv.reshape([other_cv.shape[0] // 2, 2, 1])
f_max = 25
bandwidth = 0.05
save_npy = f"{path_processed}FES_{cv_name}_path.npy"
save_fig = path_report + f"FES_{cv_name}.png"
save_extent = f"{path_processed}/extent_{cv_name}_path.npy"
spc.plot_path_vs_cv_FES(
    s_path,
    other_cv,
    weights,
    cv_name,
    cv_fig_label,
    bandwidth,
    f_max,
    save_npy,
    save_fig,
    save_extent,
)

### 0th SU

In [ ]:
cv_name = "dis_G79-Y82"
n_su = 0
cv_fig_label = f"G79-Y82 {n_su}th SU (nm)"
other_cv = np.load(f"{path_interim}/{cv_name}.npy")[:, :]
other_cv = other_cv[:, n_su : n_su + 1]
other_cv = other_cv.reshape([other_cv.shape[0] // 2, 2, 1])
f_max = 25
bandwidth = 0.05
save_npy = f"{path_processed}FES_{cv_name}_path_{n_su}.npy"
save_fig = path_report + f"FES_{cv_name}_{n_su}.png"
save_extent = f"{path_processed}/extent_{cv_name}_path.npy"
spc.plot_path_vs_cv_FES(
    s_path,
    other_cv,
    weights,
    cv_name,
    cv_fig_label,
    bandwidth,
    f_max,
    save_npy,
    save_fig,
    save_extent,
)

### 1th SU

In [ ]:
cv_name = "dis_G79-Y82"
n_su = 1
cv_fig_label = f"G79-Y82 {n_su}th SU (nm)"
other_cv = np.load(f"{path_interim}/{cv_name}.npy")[:, :]
other_cv = other_cv[:, n_su : n_su + 1]
other_cv = other_cv.reshape([other_cv.shape[0] // 2, 2, 1])
f_max = 25
bandwidth = 0.05
save_npy = f"{path_processed}FES_{cv_name}_path_{n_su}.npy"
save_fig = path_report + f"FES_{cv_name}_{n_su}.png"
save_extent = f"{path_processed}/extent_{cv_name}_path.npy"
spc.plot_path_vs_cv_FES(
    s_path,
    other_cv,
    weights,
    cv_name,
    cv_fig_label,
    bandwidth,
    f_max,
    save_npy,
    save_fig,
    save_extent,
)

### 2th SU

In [ ]:
cv_name = "dis_G79-Y82"
n_su = 2
cv_fig_label = f"G79-Y82 {n_su}th SU (nm)"
other_cv = np.load(f"{path_interim}/{cv_name}.npy")[:, :]
other_cv = other_cv[:, n_su : n_su + 1]
other_cv = other_cv.reshape([other_cv.shape[0] // 2, 2, 1])
f_max = 25
bandwidth = 0.05
save_npy = f"{path_processed}FES_{cv_name}_path_{n_su}.npy"
save_fig = path_report + f"FES_{cv_name}_{n_su}.png"
save_extent = f"{path_processed}/extent_{cv_name}_path.npy"
spc.plot_path_vs_cv_FES(
    s_path,
    other_cv,
    weights,
    cv_name,
    cv_fig_label,
    bandwidth,
    f_max,
    save_npy,
    save_fig,
    save_extent,
)

### 3th SU

In [ ]:
cv_name = "dis_G79-Y82"
n_su = 3
cv_fig_label = f"G79-Y82 {n_su}th SU (nm)"
other_cv = np.load(f"{path_interim}/{cv_name}.npy")[:, :]
other_cv = other_cv[:, n_su : n_su + 1]
other_cv = other_cv.reshape([other_cv.shape[0] // 2, 2, 1])
f_max = 25
bandwidth = 0.05
save_npy = f"{path_processed}FES_{cv_name}_path_{n_su}.npy"
save_fig = path_report + f"FES_{cv_name}_{n_su}.png"
save_extent = f"{path_processed}/extent_{cv_name}_path.npy"
spc.plot_path_vs_cv_FES(
    s_path,
    other_cv,
    weights,
    cv_name,
    cv_fig_label,
    bandwidth,
    f_max,
    save_npy,
    save_fig,
    save_extent,
)

## Path vs R64-D80

### Mean

In [ ]:
cv_name = "dis_R64-D80"
cv_fig_label = "R64-D80 (nm)"
other_cv = np.load(f"{path_interim}/{cv_name}.npy")[:, :]
other_cv = np.mean(other_cv, axis=1, keepdims=True)
other_cv = other_cv.reshape([other_cv.shape[0] // 2, 2, 1])
f_max = 25
bandwidth = 0.05
save_npy = f"{path_processed}FES_{cv_name}_path.npy"
save_fig = path_report + f"FES_{cv_name}.png"
save_extent = f"{path_processed}/extent_{cv_name}_path.npy"
spc.plot_path_vs_cv_FES(
    s_path,
    other_cv,
    weights,
    cv_name,
    cv_fig_label,
    bandwidth,
    f_max,
    save_npy,
    save_fig,
    save_extent,
)

### 0th SU

In [ ]:
cv_name = "dis_R64-D80"
n_su = 0
cv_fig_label = f"R64-D80 {n_su}th SU (nm)"
other_cv = np.load(f"{path_interim}/{cv_name}.npy")[:, :]
other_cv = other_cv[:, n_su : n_su + 1]
other_cv = other_cv.reshape([other_cv.shape[0] // 2, 2, 1])
f_max = 25
bandwidth = 0.05
save_npy = f"{path_processed}FES_{cv_name}_path_{n_su}.npy"
save_fig = path_report + f"FES_{cv_name}_{n_su}.png"
save_extent = f"{path_processed}/extent_{cv_name}_path.npy"
spc.plot_path_vs_cv_FES(
    s_path,
    other_cv,
    weights,
    cv_name,
    cv_fig_label,
    bandwidth,
    f_max,
    save_npy,
    save_fig,
    save_extent,
)

### 1th SU

In [ ]:
cv_name = "dis_R64-D80"
n_su = 1
cv_fig_label = f"R64-D80 {n_su}th SU (nm)"
other_cv = np.load(f"{path_interim}/{cv_name}.npy")[:, :]
other_cv = other_cv[:, n_su : n_su + 1]
other_cv = other_cv.reshape([other_cv.shape[0] // 2, 2, 1])
f_max = 25
bandwidth = 0.05
save_npy = f"{path_processed}FES_{cv_name}_path_{n_su}.npy"
save_fig = path_report + f"FES_{cv_name}_{n_su}.png"
save_extent = f"{path_processed}/extent_{cv_name}_path.npy"
spc.plot_path_vs_cv_FES(
    s_path,
    other_cv,
    weights,
    cv_name,
    cv_fig_label,
    bandwidth,
    f_max,
    save_npy,
    save_fig,
    save_extent,
)

### 2th SU

In [ ]:
cv_name = "dis_R64-D80"
n_su = 2
cv_fig_label = f"R64-D80 {n_su}th SU (nm)"
other_cv = np.load(f"{path_interim}/{cv_name}.npy")[:, :]
other_cv = other_cv[:, n_su : n_su + 1]
other_cv = other_cv.reshape([other_cv.shape[0] // 2, 2, 1])
f_max = 25
bandwidth = 0.05
save_npy = f"{path_processed}FES_{cv_name}_path_{n_su}.npy"
save_fig = path_report + f"FES_{cv_name}_{n_su}.png"
save_extent = f"{path_processed}/extent_{cv_name}_path.npy"
spc.plot_path_vs_cv_FES(
    s_path,
    other_cv,
    weights,
    cv_name,
    cv_fig_label,
    bandwidth,
    f_max,
    save_npy,
    save_fig,
    save_extent,
)

### 3th SU

In [ ]:
cv_name = "dis_R64-D80"
n_su = 3
cv_fig_label = f"R64-D80 {n_su}th SU (nm)"
other_cv = np.load(f"{path_interim}/{cv_name}.npy")[:, :]
other_cv = other_cv[:, n_su : n_su + 1]
other_cv = other_cv.reshape([other_cv.shape[0] // 2, 2, 1])
f_max = 25
bandwidth = 0.05
save_npy = f"{path_processed}FES_{cv_name}_path_{n_su}.npy"
save_fig = path_report + f"FES_{cv_name}_{n_su}.png"
save_extent = f"{path_processed}/extent_{cv_name}_path.npy"
spc.plot_path_vs_cv_FES(
    s_path,
    other_cv,
    weights,
    cv_name,
    cv_fig_label,
    bandwidth,
    f_max,
    save_npy,
    save_fig,
    save_extent,
)